In [200]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn

import os
from datetime import datetime
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

import uti
from uti_func import *

# MLP

## old features

In [77]:
accuracy = []
score = []
for nation in range(5):
    League = ["France","England","Germany","Italy","Spain"]
    DataPath = League[nation]
    os.chdir(DataPath)
    league_datasets= os.listdir()
    league_datasets = sorted(league_datasets)
    
    for year in range(5):
        # get one specific year
        data = pd.read_csv(league_datasets[year])
        print(league_datasets[year])
        
        # No change
        teams = data['HomeTeam'].unique()

        Model_Inputs = pd.DataFrame(columns=['Team','Opponent','H_Scores_Ave','H_Shots','H_ShotsOnTargetRates',
                                  'H_Goals','H_LossGoals','H_Corners','H_Winning_Prob',
                                  'A_Scores_Ave','A_Shots','A_ShotsOnTargetRates',
                                  'A_Goals','A_LossGoals','A_Corners','A_Winning_Prob',
                                  'GoalDif'
                                  ])

        Model_Inputs['Team']=data['HomeTeam']
        Model_Inputs['Opponent']=data['AwayTeam']
        Model_Inputs['GoalDif']=data['FTHG']-data['FTAG']

        Home_Features=['H_Scores_Ave','H_Shots','H_ShotsOnTargetRates',
                                  'H_Goals','H_LossGoals','H_Corners','H_Winning_Prob']

        Away_Features=['A_Scores_Ave','A_Shots','A_ShotsOnTargetRates',
                                  'A_Goals','A_LossGoals','A_Corners','A_Winning_Prob']

        for t in teams:

            team_data = uti.MatchResults(data,t)
            t_stats = uti.Team_Stats(team_data)
    
            # Home Team 
            Model_Inputs.loc[Model_Inputs['Team']==t,Home_Features]=np.copy(t_stats[t_stats['Home_Indicator']==1].iloc[:,2:9])
    
            # Away Team
            Model_Inputs.loc[Model_Inputs['Opponent']==t,Away_Features]=np.copy(t_stats[t_stats['Home_Indicator']==0].iloc[:,9:16])
        
        # create result result of [0 1 2], label of [H D A] 
        result = []
        label = []
        for i in range(Model_Inputs.shape[0]):
            if Model_Inputs.iloc[i,-1] > 0:
                result.append(1)
                label.append('H')
            elif Model_Inputs.iloc[i,-1] == 0:
                result.append(2)
                label.append('D')
            else:
                result.append(0)
                label.append('A')
        
        Model_Inputs['result'] = result
        Model_Inputs['label'] = label
        
        # fill na, scale
        Model_Inputs = Model_Inputs.replace(np.inf, np.nan).fillna(0)
        min_max_scaler = preprocessing.MinMaxScaler()
        df = pd.DataFrame(min_max_scaler.fit_transform(Model_Inputs.drop(['Team','Opponent','result','label','GoalDif'],axis=1)))
        df.columns = Model_Inputs.columns[2:-3]
        df['result'] = Model_Inputs['result']
        df['label'] = Model_Inputs['label']
        
        # split train & test
        match_per_round = 9 if nation == 2 else 10
        k = 5
        X_train = df.iloc[:-match_per_round*k,:-2]
        X_test = df.iloc[-match_per_round*k:,:-2]
        y_train = df.iloc[:-match_per_round*k,-1]
        y_test = df.iloc[-match_per_round*k:,-1]
        
        # train mlp
        mlp.fit(X_train,y_train)
        mlp_preds = mlp.predict_proba(X_test)
        
        accuracy.append(mlp.score(X_test,y_test))
        score.append(np.mean(Brier_Score(list(y_test),mlp_preds[:,1],mlp_preds[:,2],mlp_preds[:,0])))
        
    os.chdir("../")

print('mean accuracy:')
print(np.mean(accuracy))
print('mean score:')
print(np.mean(score))

France12.csv


/Users/ice/Documents/1003-ML/project/uti.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  HomeMatches['Full_GoalDif']= stats.loc[stats['Home_Indicator']==1,'Full_GoalDif']
/Users/ice/Documents/1003-ML/project/uti.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  AwayMatches['Full_GoalDif']= stats.loc[stats['Home_Indicator']==0,'Full_GoalDif']


France13.csv
France14.csv
France15.csv
France16.csv


/Users/ice/Documents/1003-ML/project/uti.py:172: RuntimeWarning: divide by zero encountered in true_divide
  stats.loc[team_data['Home_Indicator']==0,'A_ShotsOnTargetRates']= np.cumsum(np.array(AwayMatches['Away_Shots_on_Target'])/np.array(HomeMatches['Away_Shots']))/NumMatches


England12.csv
England13.csv
England14.csv
England15.csv
England16.csv
Germany13.csv
Germany14.csv
Germany15.csv
Germany16.csv
Germany17.csv
Italy12.csv
Italy13.csv
Italy14.csv
Italy15.csv
Italy16.csv
Spain12.csv
Spain13.csv
Spain14.csv
Spain15.csv
Spain16.csv
mean accuracy:
0.534044444444
mean score:
1.05187774482


In [76]:
os.chdir('../')

## new features

In [231]:
def dataTransformer(data,features):
    
    """
    param:
        @data: DataFrame
        @features: (list) the features need to be extracted from original dataset
    """
    
    team_lst = np.unique(data['HomeTeam'].tolist() + data['AwayTeam'].tolist())
    team_store = {team:0 for team in team_lst} 
    
    for team in team_lst:
        home_data = data[data['HomeTeam']==team]
        col_name = home_data.columns.tolist()  
        col_name.insert(col_name.index('AwayTeam')+1,'Home_Indicator')  
        home_data = home_data.reindex(columns=col_name)  
        home_data['Home_Indicator'] = 1
        home_data = home_data.rename(columns={'HomeTeam': 'Team', 'AwayTeam': 'Opponent', 
                                             'FTHG' : 'Team_Full_Goals', 'FTAG': 'Opponent_Full_Goals',
                                             'HTHG': 'Team_Half_Goals', 'HTAG': 'Opponent_Half_Goals',
                                             'HST': 'Team_Shots_On_Target', 'AST': 'Opponent_Shots_On_Target',
                                             'HS': 'Team_Shots', 'AS':'Opponent_Shots'})
        home_data['Goals'] = home_data['Team_Full_Goals']
        
        away_data = data[data['AwayTeam']==team]
        col_name_2 = away_data.columns.tolist()  
        col_name_2.insert(col_name_2.index('AwayTeam')+1,'Home_Indicator')  
        away_data = away_data.reindex(columns=col_name_2)  
        away_data['Home_Indicator'] = 0
        away_data = away_data.rename(columns={'HomeTeam': 'Opponent', 'AwayTeam': 'Team', 
                                             'FTHG' : 'Opponent_Full_Goals', 'FTAG': 'Team_Full_Goals',
                                             'HTHG': 'Opponent_Half_Goals', 'HTAG': 'Team_Half_Goals',
                                             'HST': 'Opponent_Shots_On_Target', 'AST': 'Team_Shots_On_Target',
                                             'HS': 'Opponent_Shots', 'AS':'Team_Shots'})
        away_data['Goals'] = away_data['Opponent_Full_Goals']
        
        concat_data = pd.concat([home_data, away_data])
        team_data = concat_data[features]
        
#         rename_dict = {'FTHG':'Full_Home_Goals','FTAG':'Full_Away_Goals','FTR':'Full_Results','HTHG':'Half_Home_Goals','HTAG':'Half_Away_Goals',
#                        'HST':'Home_Shots_on_Target','AST':'Away_Shots_on_Target','AS':'Away_Shots','HS':'Home_Shots'}
        rename_dict = {'FTR':'Full_Results'}
        team_data = team_data.rename(columns=rename_dict)
        
        team_data['Date'] = [datetime.strptime(x, '%d/%m/%y') for x in team_data['Date']]
        team_data = team_data.sort_values(by='Date')
        
        #compute the standing until the last game
        col_name_3 = team_data.columns.tolist()  
        col_name_3.insert(col_name_3.index('Full_Results')+1,'Standing')  
        team_data = team_data.reindex(columns=col_name_3)

        tmp = []      
        for pair in zip(team_data['Home_Indicator'], team_data['Full_Results']):
            if (pair[0]==0 and pair[1]=='A') or (pair[0]==1 and pair[1]=='H'):
                tmp.append(3)
            elif (pair[0]==1 and pair[1]=='A') or (pair[0]==0 and pair[1]=='H'):
                tmp.append(0)
            else:
                tmp.append(1)

        standing_lst = []
        for i in range(len(tmp)+2):
            if i > 1:
                standing_lst.append(np.sum(tmp[:i-1]))

        team_data['Standing'] = standing_lst
        
        #################################################################################
        # goals, shots, shots_on_target last 2-5 games
        for k in [5,4,3,2]:
            col_name_4 = team_data.columns.tolist()  
            col_name_4.insert(col_name_4.index('Team_Full_Goals')+1,'Team_Full_Goals_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('Opponent_Full_Goals')+1,'Opponent_Full_Goals_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('Team_Half_Goals')+1,'Team_Half_Goals_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('Opponent_Half_Goals')+1,'Opponent_Half_Goals_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('Team_Shots_On_Target')+1,'Team_Shots_On_Target_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('Opponent_Shots_On_Target')+1,'Opponent_Shots_On_Target_last'+str(k)+'_avg')
            team_data = team_data.reindex(columns=col_name_4)
            fthg = []
            ftag = []
            hthg = []
            htag = []
            hst = []
            ast = []
            for i in range(len(tmp)):
                if i - k + 1 < 0:
                    fthg.append(np.sum(team_data['Team_Full_Goals'][0:i + 1])/ (i + 1))
                    ftag.append(np.sum(team_data['Opponent_Full_Goals'][0:i + 1])/ (i + 1))
                    hthg.append(np.sum(team_data['Team_Half_Goals'][0:i + 1])/ (i + 1))
                    htag.append(np.sum(team_data['Opponent_Half_Goals'][0:i + 1])/ (i + 1))
                    hst.append(np.sum(team_data['Team_Shots_On_Target'][0:i + 1])/ (i + 1))
                    ast.append(np.sum(team_data['Opponent_Shots_On_Target'][0:i + 1])/ (i + 1))
                else:   
                    fthg.append(np.sum(team_data['Team_Full_Goals'][i - k + 1:i + 1])/k)
                    ftag.append(np.sum(team_data['Opponent_Full_Goals'][i - k + 1:i + 1])/k)
                    hthg.append(np.sum(team_data['Team_Half_Goals'][i - k + 1:i + 1])/k)
                    htag.append(np.sum(team_data['Opponent_Half_Goals'][i - k + 1:i + 1])/k)
                    hst.append(np.sum(team_data['Team_Shots_On_Target'][i - k + 1:i + 1])/k)
                    ast.append(np.sum(team_data['Opponent_Shots_On_Target'][i - k + 1:i + 1])/k)
            team_data['Team_Full_Goals_last'+str(k)+'_avg']= fthg
            team_data['Opponent_Full_Goals_last'+str(k)+'_avg']= ftag
            team_data['Team_Half_Goals_last'+str(k)+'_avg']= hthg
            team_data['Opponent_Half_Goals_last'+str(k)+'_avg']= htag
            team_data['Team_Shots_On_Target_last'+str(k)+'_avg']= hst
            team_data['Opponent_Shots_On_Target_last'+str(k)+'_avg']= ast
        #################################################################################
        
        #compute the winning probability
        col_name_4 = team_data.columns.tolist()  
        col_name_4.insert(col_name_4.index('Home_Indicator')+1,'Winning_Probability')
        team_data = team_data.reindex(columns=col_name_4)
        
        col_name_4 = team_data.columns.tolist()
        col_name_4.insert(col_name_4.index('Winning_Probability')+1,'Winning_Probability_last5')
        team_data = team_data.reindex(columns=col_name_4)
        
        tmp_1 = []
        for pair in zip(team_data['Home_Indicator'], team_data['Full_Results']):
            if (pair[0]==0 and pair[1]=='A') or (pair[0]==1 and pair[1]=='H'):  #win
                tmp_1.append(1)
            elif (pair[0]==1 and pair[1]=='A') or (pair[0]==0 and pair[1]=='H'): #lose
                tmp_1.append(0)
            else: #draw
                tmp_1.append(0)
        
        win_lst = []
        win_last5 = []
#         win_last5 = [0,0,0,0,0]
        for i in range(len(tmp_1)):
            win_lst.append(np.sum(tmp_1[:i + 1])/(i + 1))
        for i in range(len(tmp_1)):
            if i > 4:
                win_last5.append(np.sum(tmp_1[i-4:i + 1])/5)
            else:
                win_last5.append(np.sum(tmp_1[0 : i + 1])/ (i + 1))
        team_data['Winning_Probability']= win_lst
        team_data['Winning_Probability_last5'] = win_last5
        
        #################################################################################
        
        #compute the winning probability of being home/away team
        home_wprob_lst = []
        away_wprob_lst = []
        
        for i in range(len(team_data)):
            tmp_data = team_data[:i+1]
            tmp_df = tmp_data[tmp_data['Home_Indicator']==1]
            if len(tmp_df)==0:
                home_wprob_lst.append(0)
            else:
                home_wprob = (len(tmp_df[tmp_df['Full_Results']=='H']))/len(tmp_df)
                home_wprob_lst.append(home_wprob)

            tmp_df1 = tmp_data[tmp_data['Home_Indicator']==0]
            if len(tmp_df1)==0:
                away_wprob_lst.append(0)
            else:
                away_wprob = (len(tmp_df1[tmp_df1['Full_Results']=='A']))/len(tmp_df1)
                away_wprob_lst.append(away_wprob)
        
        col_name = team_data.columns.tolist() 
        col_name.insert(col_name.index('Winning_Probability_last5')+1,'Home_Win_Prob')
        team_data = team_data.reindex(columns=col_name)
        team_data['Home_Win_Prob'] = home_wprob_lst

        col_name = team_data.columns.tolist() 
        col_name.insert(col_name.index('Home_Win_Prob')+1,'Away_Win_Prob')
        team_data = team_data.reindex(columns=col_name)
        team_data['Away_Win_Prob'] = away_wprob_lst
        
#         rename_dict = {'FTHG':'Full_Home_Goals','FTAG':'Full_Away_Goals','HTHG':'Half_Home_Goals','HTAG':'Half_Away_Goals',
#                        'HST':'Home_Shots_on_Target','AST':'Away_Shots_on_Target','AS':'Away_Shots','HS':'Home_Shots'}
        team_data = team_data.rename(columns=rename_dict)
        
        team_store[team] = team_data
     
    return team_store

In [237]:
league_res =np.zeros((4,5))

for nation in range(5):
    League = ["France","England","Germany","Italy","Spain"]
    DataPath = League[nation]
    os.chdir(DataPath)
    league_datasets= os.listdir()
    league_datasets = sorted(league_datasets)
    evaluation_matrices = np.zeros((4,len(league_datasets)))
    
    for year in range(5):
        # get one specific year
        data = pd.read_csv(league_datasets[year]).iloc[:,1:]
        print(league_datasets[year])
        
        features_lst = ['Date','Team','Opponent', 'FTR', 'Home_Indicator','Team_Full_Goals', 'Opponent_Full_Goals',
                        'Team_Half_Goals','Opponent_Half_Goals', 'Team_Shots_On_Target', 'Opponent_Shots_On_Target','Team_Shots','Opponent_Shots',
                       'PSCH','PSCD','PSCA','Goals']
        dict_ = dataTransformer(data, features_lst)
        
        df = pd.DataFrame()
        i = 0
        for k in dict_.keys():
            i += 1
            tmp = dict_[k]
            a = tmp.iloc[:,:4].reset_index(drop = True)
            init_row = pd.DataFrame(columns=tmp.columns[4:]).reset_index(drop=True)
            init_row.loc[0] = [0]*len(tmp.columns[4:])
            b = pd.concat([init_row,tmp.iloc[:-1,4:]]).reset_index(drop=True)
            new = pd.concat([a, b],axis = 1)
            df = pd.concat([df, new], axis = 0)  
        # sort by date: split train and test by date!
        df = df.sort_values(by=['Date']).reset_index(drop = True)
        
        # create result of [0 1 2]
        df = df.loc[df['Full_Results'] != 0]
        indicator = df["Home_Indicator"].tolist()
        result = df["Full_Results"].tolist()
        
        # result: hometeam result
        
        # y: team result
        # y_predict: team result
        # y_result: team result
        # 2 draw 1 team win 0 team lose
        # D draw H team win A team lose
        y = []
        H = []
        A = []
        for i in range(len(result)):
            if indicator[i] == 1:
                H.append(df['PSCH'].tolist()[i])
                A.append(df['PSCA'].tolist()[i])
            else:
                H.append(df['PSCA'].tolist()[i])
                A.append(df['PSCH'].tolist()[i])
        df['H'] = H
        df['A'] = A   
        
        for i in range(len(result)):
            if result[i] == 'D':
                y.append(2)
            elif indicator[i] == 1:
                if result[i] == 'H':
                    y.append(1)
                else:
                    y.append(0)
            elif indicator[i] == 0:
                if result[i] == 'A':
                    y.append(1)
                else:
                    y.append(0)
                    
        # scale
        df = df.replace(np.inf, np.nan).fillna(0)
        X = df.iloc[:, 4:-6].as_matrix()
        c_max = X.max(axis = 0)
        c_min = X.min(axis = 0)
        X = (X - c_min) / (c_max - c_min)
        
        # split train & test
        match_per_round = 9 if nation == 2 else 10
        k = 5
        brier_score_mat = np.zeros((k,match_per_round))
        accuracy_list = np.zeros((k))
        cross_entropy_list = np.zeros((k))
        probability_loss = np.zeros((k))
        
        # for each league and each year, loop last k runs
        for j in range(k,0,-1): 
            # hold out the last m run as test data
            X_train = X[:-match_per_round*j,:]
            X_test = pd.DataFrame(X).tail(match_per_round*j)[:match_per_round]
            y_train = y[:-match_per_round*j]
            y_test = pd.DataFrame(y).tail(match_per_round*j)[:match_per_round]
            
            y_result = []
            for i in range(len(y_test)):
                if y_test[0].tolist()[i] == 1:
                    y_result.append('H')
                elif y_test[0].tolist()[i] == 0:
                    y_result.append('D')
                else:
                    y_result.append('A')
                    
            # train mlp
            mlp = MLPClassifier(solver='sgd',hidden_layer_sizes=(20,10),activation='relu', max_iter = 1000, shuffle = False)
            mlp.fit(X_train,y_train)
            mlp_preds = mlp.predict_proba(X_test) 
            
            real_probability = odds_transfer(df[['H','PSCD','A']])
            real_probability_mat =  real_probability.tail(match_per_round*j)[:match_per_round]
            
            # compute evaluation matric value for every road
            brier_score_mat[k-j,] = Brier_Score(y_result,mlp_preds[:,1],mlp_preds[:,2],mlp_preds[:,0])
            accuracy_list[k-j] = Accuracy(y_result,mlp_preds[:,1],mlp_preds[:,2],mlp_preds[:,0])
            cross_entropy_list[k-j] = Cross_Entropy(y_result,mlp_preds[:,1],mlp_preds[:,2],mlp_preds[:,0])
            probability_loss[k-j] = Probability_Difference(real_probability_mat,mlp_preds[:,1],mlp_preds[:,2],mlp_preds[:,0])

        # Column year, row match round
        evaluation_matrices[0,year]= np.mean(brier_score_mat)#sum(brier_score_mat.T)/matches_per_round
        evaluation_matrices[1,year]= np.mean(accuracy_list)
        evaluation_matrices[2,year]= np.mean(cross_entropy_list)
        evaluation_matrices[3,year]= np.mean(probability_loss)
        
    os.chdir("../")
    league_res[:,nation] = np.mean(evaluation_matrices,axis=1)

league_res.T
pd.DataFrame(league_res.T,index=League,columns=['Brier Score','Accuracy','Log Loss','Probability L1 Loss']) 

France12.csv


/Users/ice/Documents/1003-ML/project/uti_func.py:23: RuntimeWarning: invalid value encountered in true_divide
  nor_probas = np.divide(1/odds,np.repeat(np.sum(probas,axis=1),3).values.reshape(n,3))


France13.csv
France14.csv
France15.csv
France16.csv
England12.csv
England13.csv
England14.csv
England15.csv
England16.csv
Germany13.csv
Germany14.csv
Germany15.csv
Germany16.csv
Germany17.csv
Italy12.csv
Italy13.csv
Italy14.csv
Italy15.csv
Italy16.csv
Spain12.csv
Spain13.csv
Spain14.csv
Spain15.csv
Spain16.csv


,Brier Score,Accuracy,Log Loss,Probability L1 Loss
France,0.678552,0.332000,0.645550,0.361422
England,0.713171,0.296000,0.673412,0.401517
Germany,0.675957,0.377778,0.644909,0.340333
Italy,0.699688,0.292000,0.662436,0.410287
Spain,0.683558,0.376000,0.651618,0.454105


In [236]:
os.chdir('../')

In [154]:
pd.DataFrame(league_res.T,index=League,columns=['Brier Score','Accuracy','Log Loss','Probability L1 Loss']).to_csv("nn_results.csv")

## Improvement: add opponent features

In [9]:
# one test, two teams, compare prob to win
# p(a win) > = < p(b win)

In [ ]:
d = dict()
df1 = pd.DataFrame(columns=df.columns)
df2 = pd.DataFrame(columns=df.columns)
for i in range(df.shape[0]):
    if (df.iloc[i,0],df.iloc[i,2],df.iloc[i,1]) in d:
        df2 = df2.append(df.iloc[i])
    else:
        df1 = df1.append(df.iloc[i])
        d[(df.iloc[i,0],df.iloc[i,1],df.iloc[i,2])] = 0

df1 = df1.sort_values(['Date','Team']).reset_index(drop=True)
df2 = df2.sort_values(['Date','Opponent']).reset_index(drop=True)

In [288]:
league_res =np.zeros((4,5))

for nation in range(5):
    League = ["France","England","Germany","Italy","Spain"]
    DataPath = League[nation]
    os.chdir(DataPath)
    league_datasets= os.listdir()
    league_datasets = sorted(league_datasets)
    evaluation_matrices = np.zeros((4,len(league_datasets)))
    
    for year in range(5):
        # get one specific year
        data = pd.read_csv(league_datasets[year]).iloc[:,1:]
        print(league_datasets[year])
        
        features_lst = ['Date','Team','Opponent', 'FTR', 'Home_Indicator','Team_Full_Goals', 'Opponent_Full_Goals',
                        'Team_Half_Goals','Opponent_Half_Goals', 'Team_Shots_On_Target', 'Opponent_Shots_On_Target','Team_Shots','Opponent_Shots',
                       'PSCH','PSCD','PSCA','Goals']
        dict_ = dataTransformer(data, features_lst)
        
        df = pd.DataFrame()
        i = 0
        for k in dict_.keys():
            i += 1
            tmp = dict_[k]
            a = tmp.iloc[:,:4].reset_index(drop = True)
            tmp['Goals'] = tmp['Team_Full_Goals']
            init_row = pd.DataFrame(columns=tmp.columns[4:]).reset_index(drop=True)
            init_row.loc[0] = [0]*len(tmp.columns[4:])
            b = pd.concat([init_row,tmp.iloc[:-1,4:]]).reset_index(drop=True)
            new = pd.concat([a, b],axis = 1)
            df = pd.concat([df, new], axis = 0)  
        # sort by date: split train and test by date!
        df = df.sort_values(by=['Date']).reset_index(drop = True)

        # create result of [0 1 2]
        df = df.loc[df['Full_Results'] != 0]
        indicator = df["Home_Indicator"].tolist()
        result = df["Full_Results"].tolist()

        # result: hometeam result
        
        # y: team result
        # y_predict: team result
        # y_result: team result
        # 2 draw 1 team win 0 team lose
        # D draw H team win A team lose
        H = []
        A = []
        for i in range(len(result)):
            if indicator[i] == 1:
                H.append(df['PSCH'].tolist()[i])
                A.append(df['PSCA'].tolist()[i])
            else:
                H.append(df['PSCA'].tolist()[i])
                A.append(df['PSCH'].tolist()[i])
        df['H'] = H
        df['A'] = A   
        
                    
        # fill_na
        df = df.replace(np.inf, np.nan).fillna(0)
        
        # split data of team and opponent
        d = dict()
        df1 = pd.DataFrame(columns=df.columns)
        df2 = pd.DataFrame(columns=df.columns)
        for i in range(df.shape[0]):
            if (df.iloc[i,0],df.iloc[i,2],df.iloc[i,1]) in d:
                df2 = df2.append(df.iloc[i])
            else:
                df1 = df1.append(df.iloc[i])
                d[(df.iloc[i,0],df.iloc[i,1],df.iloc[i,2])] = 0

        df1 = df1.sort_values(['Date','Team']).reset_index(drop=True)
        df2 = df2.sort_values(['Date','Opponent']).reset_index(drop=True)
        
        # initial some parameter
        match_per_round = 9 if nation == 2 else 10
        k = 5
        brier_score_mat = np.zeros((k,match_per_round))
        accuracy_list = np.zeros((k))
        cross_entropy_list = np.zeros((k))
        probability_loss = np.zeros((k))
        
        # for each league and each year, loop last k runs
        for j in range(k,0,-1):
            
            mlpr_preds_lst = []
            result_lst = []
            real_prob_lst = []
            
            for df in [df1,df2]:
                # scale
                X = df.iloc[:, 4:-6].as_matrix()
                c_max = X.max(axis = 0)
                c_min = X.min(axis = 0)
                X = (X - c_min) / (c_max - c_min)
                
                y = df['Goals'].tolist()
                
                y_result = []
                for i in range(len(y)):
                    if y[i] == 1:
                        y_result.append('H')
                    elif y[i] == 0:
                        y_result.append('D')
                    else:
                        y_result.append('A')
                
                # split train and test
                X_train = X[:-match_per_round*j,:]
                X_test = pd.DataFrame(X).tail(match_per_round*j)[:match_per_round]
                y_train = y[:-match_per_round*j]
                y_test = pd.DataFrame(y).tail(match_per_round*j)[:match_per_round]
                y_test_result = pd.DataFrame(y_result).tail(match_per_round*j)[:match_per_round]
                result_lst.append(y_test_result)
                    
                # train mlp for goals
                mlpr = MLPRegressor(solver='sgd',hidden_layer_sizes=(20,10),activation='relu', max_iter = 1000, shuffle = False)
                mlpr.fit(X_train,y_train)
                mlpr_preds = mlpr.predict(X_test)
                mlpr_preds_lst.append(mlpr_preds)
            
                real_probability = odds_transfer(df[['H','PSCD','A']])
                real_probability_mat =  real_probability.tail(match_per_round*j)[:match_per_round]
                real_prob_lst.append(real_probability_mat)
            
            # train mlp for probabilities of team in df2
            mlpc = MLPClassifier(solver='sgd',hidden_layer_sizes=(20,10),activation='relu', max_iter = 1000, shuffle = False)
            mlpc.fit(np.array([df1['Goals'].tolist(),df2['Goals'].tolist()]).T,y_result)
            mlpc_preds = mlpc.predict_proba(np.array(mlpr_preds_lst).T)
                
            # compute evaluation matric value for every road
            brier_score_mat[k-j,] = Brier_Score(y_test_result,mlpc_preds[:,2],mlpc_preds[:,1],mlpc_preds[:,0])
            accuracy_list[k-j] = Accuracy(y_test_result,mlpc_preds[:,2],mlpc_preds[:,1],mlpc_preds[:,0])
            cross_entropy_list[k-j] = Cross_Entropy(y_test_result,mlpc_preds[:,2],mlpc_preds[:,1],mlpc_preds[:,0])
            probability_loss[k-j] = Probability_Difference(real_prob_lst[1],mlpc_preds[:,2],mlpc_preds[:,1],mlpc_preds[:,0])

        # Column year, row match round
        evaluation_matrices[0,year]= np.mean(brier_score_mat)#sum(brier_score_mat.T)/matches_per_round
        evaluation_matrices[1,year]= np.mean(accuracy_list)
        evaluation_matrices[2,year]= np.mean(cross_entropy_list)
        evaluation_matrices[3,year]= np.mean(probability_loss)
        
    os.chdir("../")
    league_res[:,nation] = np.mean(evaluation_matrices,axis=1)

league_res.T
pd.DataFrame(league_res.T,index=League,columns=['Brier Score','Accuracy','Log Loss','Probability L1 Loss']) 

France12.csv


/Users/ice/Documents/1003-ML/project/uti_func.py:23: RuntimeWarning: invalid value encountered in true_divide
  nor_probas = np.divide(1/odds,np.repeat(np.sum(probas,axis=1),3).values.reshape(n,3))


France13.csv
France14.csv
France15.csv


/Users/ice/anaconda/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


France16.csv
England12.csv
England13.csv
England14.csv
England15.csv
England16.csv
Germany13.csv
Germany14.csv
Germany15.csv
Germany16.csv
Germany17.csv
Italy12.csv
Italy13.csv
Italy14.csv
Italy15.csv
Italy16.csv
Spain12.csv
Spain13.csv
Spain14.csv
Spain15.csv
Spain16.csv


,Brier Score,Accuracy,Log Loss,Probability L1 Loss
France,0.246376,0.840000,0.344287,1.156317
England,0.336276,0.784000,0.496983,1.133634
Germany,0.267590,0.835556,0.456501,1.176722
Italy,0.329954,0.780000,0.460893,1.173280
Spain,0.103517,0.948000,0.160942,1.226868


In [287]:
os.chdir('../')

In [301]:
pd.DataFrame(league_res.T,index=League,columns=['Brier Score','Accuracy','Log Loss','Probability L1 Loss']).to_csv("nn_new_results.csv")